In [ ]:
''' This project is to predict whether a patient has ckd or not.'''

# About Data

Chronic kidney disease is the Longstanding disease of the kidneys leading to renal failure.

The kidneys filter waste and excess fluid from the blood. As kidneys fail, waste builds up.
Symptoms develop slowly and aren't specific to the disease. Some people have no symptoms at all and are diagnosed by a lab test.
Medication helps manage symptoms. In later stages, filtering the blood with a machine (dialysis) or a transplant may be required

Source : https://www.kaggle.com/mansoordaku/ckdisease 

### Attribute Information:

We use 24  medical features + 1 class = 25 ( 11 numeric ,14 nominal)

1.Age(numerical)
age in years

2.Blood Pressure(numerical)
bp in mm/Hg

3.Specific Gravity(nominal)
sg - (1.005,1.010,1.015,1.020,1.025)

4.Albumin(nominal)
al - (0,1,2,3,4,5)

5.Sugar(nominal)
su - (0,1,2,3,4,5)

6.Red Blood Cells(nominal)
rbc - (normal,abnormal)

7.Pus Cell (nominal)
pc - (normal,abnormal)

8.Pus Cell clumps(nominal)
pcc - (present,notpresent)

9.Bacteria(nominal)
ba - (present,notpresent)

10.Blood Glucose Random(numerical)
bgr in mgs/dl

11.Blood Urea(numerical)
bu in mgs/dl

12.Serum Creatinine(numerical)
sc in mgs/dl

13.Sodium(numerical)
sod in mEq/L

14.Potassium(numerical)
pot in mEq/L

15.Hemoglobin(numerical)
hemo in gms

16.Packed Cell Volume(numerical)

17.White Blood Cell Count(numerical)
wc in cells/cumm

18.Red Blood Cell Count(numerical)
rc in millions/cmm

19.Hypertension(nominal)
htn - (yes,no)

20.Diabetes Mellitus(nominal)
dm - (yes,no)

21.Coronary Artery Disease(nominal)
cad - (yes,no)

22.Appetite(nominal)
appet - (good,poor)

23.Pedal Edema(nominal)
pe - (yes,no)

24.Anemia(nominal)
ane - (yes,no)

25.Class (nominal)
class - (ckd,notckd)



# **Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix,accuracy_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Importing the Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/ckdisease/kidney_disease.csv')

In [ ]:
### Run this to Profile data

import pandas_profiling as pp

profile = pp.ProfileReport(    df, title="Chronic Kidney Disease Profile", html={"style": {"full_width": True}}, sort=None)
profile

# 2 Data Cleaning

## a. Renaming the columns in the data to their expanded form

In [ ]:
columns=pd.read_csv('../input/ckd-input/data_description.txt',sep='-')
columns=columns.reset_index()
columns.columns=['cols','abb_col_names']
df.columns=columns['abb_col_names'].values


## b. Correcting datatype of the columns

In [ ]:
df.info()

In [ ]:
features=['red blood cell count','packed cell volume','white blood cell count']
def convert_dtype(df,feature):
    df[feature] = pd.to_numeric(df[feature], errors='coerce')
    
for feature in features:
    convert_dtype(df,feature)
    
df.dtypes

## c. Drop the id column

In [ ]:
df.drop(["id"],axis=1,inplace=True) 

## d. Cleaning Categorical Columns


In [ ]:
cat_col=[col for col in df.columns if df[col].dtype=='object']
for col in cat_col:
    print('{} has {} values '.format(col,df[col].unique()))
    print('\n')

In [ ]:
#Replace incorrect values

df['diabetes mellitus'].replace(to_replace = {'\tno':'no','\tyes':'yes',' yes':'yes'},inplace=True)

df['coronary artery disease'] = df['coronary artery disease'].replace(to_replace = '\tno', value='no')

df['class'] = df['class'].replace(to_replace = 'ckd\t', value = 'ckd')



for col in cat_col:
    print('{} has {} values  '.format(col, df[col].unique()))
    print('\n')

## e. Assigning random values to missing(NaN) Values

In [ ]:
data=df.copy()

### Categorical features using Random value Imputation & Mode imputation

In [ ]:
data[cat_col].isnull().sum()

#### Random_value_imputation for "red blood cells" & "pus cell"


In [ ]:
def Random_value_imputation(feature):
    random_sample=data[feature].dropna().sample(data[feature].isnull().sum())               
    random_sample.index=data[data[feature].isnull()].index
    data.loc[data[feature].isnull(),feature]=random_sample

Random_value_imputation('pus cell')
Random_value_imputation('red blood cells')
data[cat_col].isnull().sum()

#### mode_imputation for all other categorical features

In [ ]:
def mode_imputation(feature):
    mode=data[feature].mode()[0]
    data[feature]=data[feature].fillna(mode)
    
for col in cat_col:
    mode_imputation(col)
    
data[cat_col].isnull().sum()

### Numerical features using Random value Imputation

In [ ]:
num_col=[col for col in df.columns if df[col].dtype!='object']
data[num_col].isnull().sum()

In [ ]:
for col in num_col:
    Random_value_imputation(col)
    
data[num_col].isnull().sum()

# 3. Data Visualization

###  label distribution for Class data

In [ ]:
sns.countplot(x='class',data=data)
plt.xlabel("class")
plt.ylabel("Count")
plt.title("target Class")

###  Label distribution for Numerical data

In [ ]:
plt.figure(figsize=(30,20))
for i,feature in enumerate(num_col):
    plt.subplot(5,3,i+1)
    data[feature].hist()
    plt.title(feature)

    Observations:
        1.age looks a bit left skewed
        2.Blood gluscose random is right skewed
        3.Blood Urea is also a bit right skewed
        4.Rest of the features are lightly skewed

###  Label distribution for Categorical data

In [ ]:
plt.figure(figsize=(20,20))
for i,feature in enumerate(cat_col):
    plt.subplot(4,3,i+1)
    sns.countplot(data[feature])
    

    A few features has imbalanced categories. Stratified folds will be necessary while cross validation.

### Correlations

In [ ]:
plt.figure(figsize=(10,8))
corr_df = data.corr()
sns.heatmap(corr_df,annot=True)

    Positive Correlation:
    Specific gravity -> Red blood cell count, Packed cell volume and Hemoglobin
    Sugar -> Blood glucose random
    Blood Urea -> Serum creatinine
    Hemoglobin -> Red Blood cell count <- packed cell volume
    
    Negative Correlation:
    Albumin, Blood urea -> Red blood cell count, packed cell volume, Hemoglobin
    Serum creatinine -> Sodium

In [ ]:
data.groupby(['red blood cells','class'])['red blood cell count'].agg(['count','mean','median','min','max'])

### Let's check for Positive correlation and its impact on classes¶

In [ ]:
px.violin(data,y='red blood cell count',x="class", color="class")

In [ ]:
px.scatter(df,'haemoglobin','packed cell volume')

### analysing distribution of 'red_blood_cell_count' in both Labels 

In [ ]:
grid=sns.FacetGrid(df, hue="class",aspect=2)
grid.map(sns.kdeplot, 'red blood cell count')
grid.add_legend()

    Both distributions are quite different, distribution CKD is quite normal and evenly distributed but not CKD distribution is a little bit left-skewed but quite close to a normal distribution

### Defining violin and scatter plot & kde_plot functions

In [ ]:
def violin(col):
    fig = px.violin(data, y=col, x="class", color="class", box=True)
    return fig.show()

def scatters(col1,col2):
    fig = px.scatter(data, x=col1, y=col2, color="class")
    return fig.show()

def kde_plot(feature):
    grid = sns.FacetGrid(data, hue="class",aspect=2)
    grid.map(sns.kdeplot, feature)
    grid.add_legend()

In [ ]:
kde_plot('red blood cell count')

In [ ]:
kde_plot('haemoglobin')

In [ ]:
scatters('red blood cell count', 'packed cell volume')

In [ ]:
scatters('red blood cell count', 'haemoglobin')

In [ ]:
scatters('haemoglobin','packed cell volume')

    1.RBC count range ~2 to <4.5 and Hemoglobin between 3 to <13 are mostly classified as positive for chronic kidney  
    disease(i.e ckd).
    2.RBC count range >4.5 to ~6.1 and Hemoglobin between >13 to 17.8 are classified as negative for chronic kidney 
    disease(i.e nockd).

In [ ]:
violin('red blood cell count')

In [ ]:
violin('packed cell volume')

###  Now let's check for negative correlation and its impact on classes

    Albumin, Blood urea -> Red blood cell count, packed cell volume, Haemoglobin

In [ ]:
scatters('red blood cell count','albumin')

    Clearly, albumin levels of above 0 affect ckd largely

In [ ]:
scatters('packed cell volume','blood urea')

    Packed cell volume >= 40 largely affects to be non ckd

In [ ]:
fig = px.bar(data, x="specific gravity", y="packed cell volume",
             color='class', barmode='group',
             height=400)
fig.show()

   

 Clearly, specific gravity >=1.02 affects non ckd

# 4. Feature Encoding / Label Encoder

In [ ]:
for col in cat_col:
    print('{} categories in the column- {} '.format(data[col].nunique(), col))
    

#### as we have just 2 categories in each feature then we can consider Label Encoder as it will not cause Curse of Dimensionality

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
for col in cat_col:
    data[col]=le.fit_transform(data[col])
    
data.head()

# 5. Feature Selection

chi2-Internally this class is going to check that whether p-value is less than 0.05 or not based on that,it will order all the features

In [ ]:
from sklearn.feature_selection import SelectKBest #Also known as Information Gain
from sklearn.feature_selection import chi2

ind_col=[col for col in data.columns if col!='class']
dep_col='class'

x=data[ind_col]
y=data[dep_col] 

ordered_rank_features=SelectKBest(score_func=chi2,k=20)
ordered_feature=ordered_rank_features.fit(x,y)

datascores=pd.DataFrame(ordered_feature.scores_,columns=["Score"])

dfcolumns=pd.DataFrame(x.columns)

features_rank=pd.concat([dfcolumns,datascores],axis=1)

features_rank.columns=['Features','Score']

# features_rank.nlargest(10,'Score')  --->  fetch largest 10 values of Score column   

selected_columns=features_rank.nlargest(10,'Score')['Features'].values
X_new=data[selected_columns]
X_new

# 6. Model

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, train_size=0.7, random_state=123)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import lightgbm
import xgboost
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

names = [
    "CatBoostClassifier",
    "Support Vector Machine",
    "Decision Tree",
    "Neural Network",
    "Random Forest",
    "XGBoost",
    "LGBMClassifier",
    "XGBRFClassifier",
    "GradientBoosting",
    "GaussianNB",
    "KNeighborsClassifier"
]
models = [
    CatBoostClassifier(verbose= False),
    SVC(),
    DecisionTreeClassifier(),
    MLPClassifier(),
    RandomForestClassifier(),
    XGBClassifier(),
    lightgbm.LGBMClassifier(max_depth=2, random_state=4),
    xgboost.XGBRFClassifier(max_depth=3, random_state=1),
    GradientBoostingClassifier(max_depth=2, random_state=1),
    GaussianNB(),
    KNeighborsClassifier(n_neighbors=5, p=2, metric='minkowski')
]

accuracy=[]

In [ ]:
for model, name in zip(models,names):
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print('Confusion matrix of ',name)
    print(confusion_matrix(y_test, y_pred))
    ac = accuracy_score(y_test, y_pred)
    print('Accuracy score is ',ac)
    accuracy.append(ac)
    print('='*50)

Accuracy_list = pd.DataFrame(list(zip(names, accuracy)),columns =['Model', 'Accuracy'])
Accuracy_list= Accuracy_list.sort_values('Accuracy', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last', ignore_index=True, key=None)

plt.rcParams['figure.figsize']=20,6 
sns.set_style("darkgrid")
ax = sns.barplot(x = 'Model',y = 'Accuracy',data = Accuracy_list , palette = "rocket", saturation =1.5)
plt.xlabel("Model", fontsize = 20 )
plt.ylabel("Accuracy", fontsize = 20)
plt.title("Sorted Accuracy of different Models", fontsize = 20)
plt.xticks(fontsize = 11, horizontalalignment = 'center', rotation = 8)
plt.yticks(fontsize = 13)
for p in ax.patches:
    width, height = p.get_width(), p.get_height()
    x, y = p.get_xy() 
    ax.annotate(f'{height:.2%}', (x + width/2, y + height*1.02), ha='center', fontsize = 'x-large')
plt.show()

 # Please let me know in the comments what improvements can be done 